In [1]:
import import_ipynb
import c1_extract_paragraphe_issue as c1 
import c2_extract_sentence_issue as c2 
import g1_generate_dictionary as g1
import pandas as pd


importing Jupyter notebook from c1_extract_paragraphe_issue.ipynb
Issue  34
importing Jupyter notebook from c2_extract_sentence_issue.ipynb
importing Jupyter notebook from g1_generate_dictionary.ipynb


In [2]:
DICTIONARY = g1.compute_dictionary()
n = 0
DICTIONARY_NUM = dict()
for k in DICTIONARY:

    DICTIONARY_NUM[k] = n 
    n +=1
NAMES = []
for x in DICTIONARY.values():
    if(len(x) == 1):
        NAMES.append(x[0])

    else:
        NAMES += [k for k in x]

n = 0
DICTIONARY_NUM = dict()
for k in DICTIONARY:

    DICTIONARY_NUM[k] = n 
    n +=1
NAMES = []
for x in DICTIONARY.values():
    if(len(x) == 1):
        NAMES.append(x[0])

    else:
        NAMES += [k for k in x]

In [3]:
def create_df(cooperations,issue_number):

    ca = [x[0] for x in cooperations]
    cb = [x[1] for x in cooperations]
    id_ca = [x[17] for x in cooperations]
    id_cb = [x[18] for x in cooperations]
    behalf = [int(float(x[5])) for x in cooperations]
    support = [int(float(x[6]))for x in cooperations]
    agreement =[int(float(x[8]))for x in cooperations]
    opposition =[int(float(x[10]))for x in cooperations]
    criticism =[int(float(x[11])) for x in cooperations]
    cooperation =[int(float(x[12])) for x in cooperations]
    sentences = [x[-3] for x in cooperations]
    dict_issue = {'type':'original','issue': issue_number,'id_ca':id_ca,'id_cb':id_cb,'Country A':ca, 'Country B': cb, 'behalf':behalf,'support':support,'agreement':agreement,'opposition':opposition,'criticism':criticism,'cooperation':cooperation, 'sentences':sentences}
    df = pd.DataFrame(dict_issue)
    return df

In [4]:
def extract_relationships_count(issue_number):
    lines = []
    sentences = []
    data = []
    entities = []
    with open("Text/ENB_relationships.txt") as infile:
        k=0
        for line in infile:
            x = line.replace('"','').split('\t')[:26]
            entities.append(x[4])
            entities.append(x[3])
            if(k > 0 and int(x[23]) == issue_number and k!=0):
                lines.append(line)
            k += 1
    for l in lines:
        list_line = l.replace('"','').split('\t')[:26]
            
        
        informations = list_line[3:19]
        informations = [f.upper() for f in informations]
        se_new = list_line[25]
        if(se_new != ''):

            se_old = se_new
            sentences.append(se_old)
            informations.append(se_old)
        else:

            sentences.append(se_old)
            informations.append(se_old)
        id_cb = 8888
        id_ca = 8888
        #print('Countries :',list_line[3],list_line[4])
        if(list_line[3] in DICTIONARY_NUM.keys()):
            id_ca = DICTIONARY_NUM[list_line[3]]

        if(list_line[4] in DICTIONARY_NUM.keys()):
            id_cb = DICTIONARY_NUM[list_line[4]]

        if(id_cb == 8888 or id_ca == 8888):
            print(id)
        informations.append(id_ca)
        informations.append(id_cb)
        data.append(informations)

    return create_df(data,issue_number), sentences

In [5]:
d , s = extract_relationships_count(505)